<a href="https://colab.research.google.com/github/nickt573/Google-Colab-Research/blob/main/image%20classifier/pneumonia_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
!pip install matplotlib torch torchvision kagglehub

# Managing Data

In [ ]:
import os
import kagglehub
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),           # Safe for X-rays
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = datasets.ImageFolder(os.path.join(path, "chest_xray/train"), train_transform)
val_dataset = datasets.ImageFolder(os.path.join(path, "chest_xray/test"), val_transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

# Building model

Reset all parameters

In [ ]:
import torch
from pneumonia import Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

history = {}
history["train_acc"] = []
history["val_acc"] = []
model = Model().to(device)
print(device)

# Running model

In [ ]:
epoch = 10
for i in range(epoch):
    print(f"Epoch {i + 1}/{epoch}")
    acc = model.train_model(train_loader)
    history["train_acc"].append(acc)
    acc = model.test_model(val_loader, False)
    history["val_acc"].append(acc)

# Visualizing model

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(history["train_acc"]) + 1)

plt.figure(figsize=(8, 5))
plt.plot(epochs, history["train_acc"], label='Train Accuracy', marker='o')
plt.plot(epochs, history["val_acc"], label='Validation Accuracy', marker='s')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
_ = model.test_model(val_loader, True)